In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [2]:
import open3d as o3d
import numpy as np

from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from optimiser_weighted import inference_weighted

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

DATA_PATH = "/scratchdata/indoor_lift"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/dust3r/dust3r/cloud_opt/base_opt.py:275: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/dust3r/dust3r/cloud_opt/plane_optimizer.py:115: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [3]:
model_name = "/scratchdata/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth"
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

/dust3r/dust3r/model.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location='cpu')


... loading model from /scratchdata/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), enc_embed_dim=1024, enc_depth=24, enc_num_heads=16, dec_embed_dim=768, dec_depth=12, dec_num_heads=12, freeze='encoder', landscape_only=False)
<All keys matched successfully>


In [4]:
images_array = []

for i in range(0,1858,30):
    images_array.append(os.path.join(DATA_PATH,"rgb/{}.png".format(i)))
images = load_images(images_array, size=512, verbose=True)

>> Loading a list of 62 images
 - adding /scratchdata/indoor_lift/rgb/0.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/30.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/60.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/90.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/120.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/150.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/180.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/210.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/240.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/270.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/300.png with resolution 640x400 --> 512x320


 - adding /scratchdata/indoor_lift/rgb/330.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/360.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/390.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/420.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/450.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/480.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/510.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/540.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/570.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/600.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/630.png with resolution 640x400 --> 512x320
 - adding /scratchdata/indoor_lift/rgb/660.png with resolution 640x400 --> 512x320
 - a

In [5]:
pairs = make_pairs(images, scene_graph='swin-3', prefilter=None, symmetrize=True)
output = inference(pairs, model, device, batch_size=batch_size)

>> Inference with model on 372 image pairs


  0%|          | 0/372 [00:00<?, ?it/s]/dust3r/dust3r/inference.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=bool(use_amp)):
/dust3r/dust3r/model.py:205: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/dust3r/dust3r/inference.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
100%|██████████| 372/372 [01:31<00:00,  4.07it/s]


In [6]:
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

 init edge (2*,4*) score=265.015869140625
 init edge (1*,4) score=257.3829650878906
 init edge (1,3*) score=255.63870239257812
 init edge (1,61*) score=252.15870666503906
 init edge (2,5*) score=242.8963165283203
 init edge (4,6*) score=232.4735870361328
 init edge (4,7*) score=189.1025848388672
 init edge (5,8*) score=168.43032836914062
 init edge (6,9*) score=113.43773651123047
 init edge (8,10*) score=53.9981575012207
 init edge (60*,61) score=261.3708801269531
 init edge (0*,61) score=252.37081909179688
 init edge (59*,60) score=247.49317932128906
 init edge (58*,59) score=225.33670043945312
 init edge (57*,58) score=145.34097290039062
 init edge (56*,57) score=179.1665802001953
 init edge (55*,56) score=131.88539123535156
 init edge (54*,55) score=143.6730499267578
 init edge (53*,54) score=143.48680114746094
 init edge (52*,53) score=131.4656219482422
 init edge (50*,52) score=115.68419647216797
 init edge (49*,50) score=87.21581268310547
 init edge (50,51*) score=135.21656799316

100%|██████████| 300/300 [00:59<00:00,  5.05it/s, lr=1.27413e-06 loss=0.0292659]


In [7]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

In [12]:
print(poses[0])

tensor([[ 9.6646e-01,  2.3566e-04,  2.5682e-01,  5.5775e-03],
        [-9.2691e-03,  9.9938e-01,  3.3964e-02,  2.5058e-02],
        [-2.5665e-01, -3.5205e-02,  9.6586e-01, -1.2422e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [30]:
i = 0
pointcloud = pts3d[i].detach().cpu().numpy()
pointcloud = pointcloud.reshape(-1, 3)
color = imgs[i].reshape(-1, 3)
pose = poses[i].detach().cpu().numpy()

dist = np.linalg.norm(pointcloud - pose[:3,3], axis=1)
norm_dist = (dist-dist.min())/(dist.max()-dist.min())
mask = norm_dist < 0.5

print(pointcloud[mask])


[[-0.10820186 -0.0692589   0.08428344]
 [-0.1075123  -0.06920169  0.08398107]
 [-0.1075325  -0.06973861  0.08499013]
 ...
 [ 0.22235408  0.13979113  0.00493105]
 [ 0.22595738  0.14133254  0.00649323]
 [ 0.22105752  0.13839196  0.00301899]]


In [31]:
pcd = o3d.geometry.PointCloud()

for i in range(len(pts3d)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    pose = poses[i].detach().cpu().numpy()

    dist = np.linalg.norm(pointcloud - pose[:3,3], axis=1)
    norm_dist = (dist-dist.min())/(dist.max()-dist.min())
    mask = norm_dist < 0.5
    
    tmp_pcd = o3d.geometry.PointCloud()
    tmp_pcd.points = o3d.utility.Vector3dVector(pointcloud[mask])
    tmp_pcd.colors = o3d.utility.Vector3dVector(color[mask])
    
    pcd += tmp_pcd
    pcd = pcd.voxel_down_sample(0.01)
    
#o3d.visualization.draw_geometries([pcd])

In [32]:
# Save o3d
o3d.io.write_point_cloud("tmp_1.ply", pcd)

True